In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [20]:
import sys
from pathlib import Path

# Add the src directory to the Python path
sys.path.append(str(Path().resolve().parent / 'src'))

# Now you can import the clean_data function
from data_cleaning import drop_columns, impute_missing_values
from data_preprocessing import preprocess_data

data = pd.read_csv("../input/train_folds.csv")
df = drop_columns(data)

In [21]:
for fold in range(5):
    fold_df = impute_missing_values(df, fold)
    fold_df = impute_missing_values(df, 0)
    fold_df = preprocess_data(fold_df, fold=0)
    # drop the kfold column
    fold_df = fold_df.drop("kfold", axis=1)

    # Split the data into X and y
    X = fold_df.drop("Converted", axis=1)
    y = fold_df["Converted"]
    # Split the data into test and train
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the logistic regression model
    log_reg = LogisticRegression()
    log_reg.fit(X_train, y_train)

    # Make predictions
    y_pred = log_reg.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    print(f"Classification Report: {classification_report(y_test, y_pred)}")
    print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")

c:\Users\Suraj\AppData\Local\pypoetry\Cache\virtualenvs\lead_scoring-r0dzUjaH-py3.12\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Suraj\AppData\Local\pypoetry\Cache\virtualenvs\lead_scoring-r0dzUjaH-py3.12\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Accuracy: 0.797077922077922
Classification Report:               precision    recall  f1-score   support

           0       0.82      0.86      0.84      1150
           1       0.75      0.70      0.72       698

    accuracy                           0.80      1848
   macro avg       0.79      0.78      0.78      1848
weighted avg       0.80      0.80      0.80      1848

Confusion Matrix: [[985 165]
 [210 488]]
Accuracy: 0.797077922077922
Classification Report:               precision    recall  f1-score   support

           0       0.82      0.86      0.84      1150
           1       0.75      0.70      0.72       698

    accuracy                           0.80      1848
   macro avg       0.79      0.78      0.78      1848
weighted avg       0.80      0.80      0.80      1848

Confusion Matrix: [[985 165]
 [210 488]]


c:\Users\Suraj\AppData\Local\pypoetry\Cache\virtualenvs\lead_scoring-r0dzUjaH-py3.12\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Suraj\AppData\Local\pypoetry\Cache\virtualenvs\lead_scoring-r0dzUjaH-py3.12\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Accuracy: 0.797077922077922
Classification Report:               precision    recall  f1-score   support

           0       0.82      0.86      0.84      1150
           1       0.75      0.70      0.72       698

    accuracy                           0.80      1848
   macro avg       0.79      0.78      0.78      1848
weighted avg       0.80      0.80      0.80      1848

Confusion Matrix: [[985 165]
 [210 488]]
Accuracy: 0.797077922077922
Classification Report:               precision    recall  f1-score   support

           0       0.82      0.86      0.84      1150
           1       0.75      0.70      0.72       698

    accuracy                           0.80      1848
   macro avg       0.79      0.78      0.78      1848
weighted avg       0.80      0.80      0.80      1848

Confusion Matrix: [[985 165]
 [210 488]]
Accuracy: 0.797077922077922
Classification Report:               precision    recall  f1-score   support

           0       0.82      0.86      0.84      1150
  

c:\Users\Suraj\AppData\Local\pypoetry\Cache\virtualenvs\lead_scoring-r0dzUjaH-py3.12\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
